In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from decimal import Decimal


# Define colors globally for use across all plots
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEEAD', '#D4A5A5']

def plot_summary_charts(df, df_name):
    # Convert any Decimal values to float for all numerical columns
    numerical_cols = ['heart_rate', 'mbp', 'resp_rate', 'spo2', 'age', 'icu_los_hours', 'hosp_los_hours']
    for col in numerical_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Define columns
    categorical_cols = ['gender', 'has_hypertension', 'has_hydrocephalus', 'has_cad', 
                       'has_anticoagulation', 'had_neurosurgery', 'gcs_verbal', 'gcs_motor', 'gcs_eyes', 'readmitted']
    numerical_cols = ['heart_rate', 'mbp', 'resp_rate', 'spo2', 'age', 'icu_los_hours']
    categorical_cols = [col for col in categorical_cols if col in df.columns]
    numerical_cols = [col for col in numerical_cols if col in df.columns]

    # Calculate number of subplots
    num_categorical = len(categorical_cols)
    num_numerical = len(numerical_cols)
    total_plots = num_categorical + num_numerical + 1  # +1 for missing values
    ncols = 3  # 3 columns for layout
    nrows = (total_plots + ncols - 1) // ncols  # Calculate rows needed

    # Create figure with subplots
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5 * nrows))
    axes = axes.flatten()  # Flatten for easier indexing
    plt.suptitle(f'Summary of {df_name}', fontsize=16, y=1.02)

    # Counter for subplot index
    plot_idx = 0

    # 1. Pie Charts for Categorical/Binary Columns
    for col in categorical_cols:
        if col in df.columns:
            value_counts = df[col].value_counts(dropna=False)
            labels = [str(x) if pd.notna(x) else 'Missing' for x in value_counts.index]
            sizes = value_counts.values

            axes[plot_idx].pie(sizes, labels=labels, colors=colors[:len(labels)], 
                              autopct='%1.1f%%', startangle=90, textprops={'fontsize': 10})
            axes[plot_idx].set_title(f'{col}', fontsize=12)
            axes[plot_idx].axis('equal')
            
            plot_idx += 1

    # 2. Bar Chart for Missing Values
    missing = df.isnull().sum()
    missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
    missing_pct = missing_pct.astype(float)  # Ensure float type
    missing_data = pd.DataFrame({'Column': missing.index, 'Missing %': missing_pct})
    missing_data = missing_data[missing_data['Missing %'] > 0]

    if not missing_data.empty:
        axes[plot_idx].bar(missing_data['Column'], missing_data['Missing %'], color=colors[0])
        axes[plot_idx].set_title('Missing Values (%)', fontsize=12)
        axes[plot_idx].set_xlabel('Column', fontsize=10)
        axes[plot_idx].set_ylabel('Percentage Missing', fontsize=10)
        axes[plot_idx].tick_params(axis='x', rotation=45, labelsize=8)
    
        
        plot_idx += 1

    # 3. Histograms for Numerical Parameters
    for col in numerical_cols:
        if col in df.columns:
            data = df[col].dropna().astype(float)  # Convert to float
            if data.empty:
                axes[plot_idx].text(0.5, 0.5, f'No data for {col}', ha='center', va='center', fontsize=10)
                axes[plot_idx].set_title(f'{col}', fontsize=12)
            else:
                axes[plot_idx].hist(data, bins=30, color=colors[1], edgecolor='black')
                axes[plot_idx].set_title(f'{col}', fontsize=12)
                axes[plot_idx].set_xlabel(col, fontsize=10)
                axes[plot_idx].set_ylabel('Count', fontsize=10)
            
            
            plot_idx += 1

    # Remove empty subplots
    for i in range(plot_idx, len(axes)):
        fig.delaxes(axes[i])

    # Adjust layout
    plt.tight_layout()
    plt.show()

plot_summary_charts(df, "df")

df